# LBA (2021 Fall, CS146)
## Jeong woo Park (M23)

### Questions to Answer
- What is the basic average price for each product? You need to think carefully about how to anchor the basic price for each product since this will depend on the currency used as well as the distribution of prices. 
- How much does each of the following factors modify the basic price of the product(up or down)? Factors: The geographical location (Country) of the grocery store, Brand of the grocery store(cheap? Mid-range? Luxury?) Explain in your report how strong each of these effects is. Which has the greatest influence on price variation between shops?
- Does price variation by geographical location correlate with variation in rental prices, or not?


The Project using Stan in Python aims to model the relevance of the prices of 10 goods and the rent. Here, the students of CS146 has gathered the data on the city that they are located in. The results of this project allows us to make predictions about the costs in the products given the rent rate of the area. 

### Justifications for the Project
- The posterior parameters should be 1) the base price for each of the product, 2) the rent of the country, 3) the store type(cheap, mid-ranged, luxury). 
- To create these three parameters, we need the three models that is related with these parameters.
- About the Base Price of the product: It's values are a positive real number. The prices will be converted to USD to make the models more easier to work with, and make the other areas available for comparison. The values will have extremes, but I am going to consider the expected value to work with the parameters. 
- The models for the base price are exponential distribution with lambda, and lambda is from Gamma distribution of (1, expected value)
- Rent price and Store type variables are both positive real numbers as I am thinking of considering as a value to multiply on the expected base value to make predictions. So, both of them will follow the Log-Normal distribution. The two variables are there to represent the ratio to multiply on the base price predictions
- Likelihood function: Normal Distribution. Since I am using exponential function for the prior, to make calculations easy as well as it is convienient to assume that the price for the food will follow normal distributions but tailored only to have positive real number values. Thus, Price ~N(RentRate variable* Storetype Variable* Base Price variable, variance)
- For hyperparameter for the prior which depends on a gamma distribution, I chose the variables to have alpha =1 and beta =1 to let the data to drive the result (allowing any positive real values to be possible for the ratio for exponential function)
- Data Processing: I disregarded Korea data as all the countries are in Europe, and Korea is very far away with distance and culture from both Berlin or London. I will convert the prices for 1kg(or 1liter) for the goods all converted into USD currency. Store types will be in 3 types as given, and NAN values are dropped to make the computation easy and keep it simple to use what I've learned only. 

### Data Pre-processing
I need to reform the data to make it fit to the Stan and thus the computer to do the math and magic. To do so, we need to have the idea below.

- Drop unnecessary rows, columns, and NAs
- Convert the prices to one currency while the goods have the same quantity for comparison. 
- Considering the difference in culture of the East and the West, and the system having more than the rent fee of the area to decide the good's prices, I dropped the data from Korea. Mainly because I thought the hypothesis was too broad to generalize by given data considering all the variables that we don't have data with especially when we only have the data from only a few countries that does not represent the whole world correctly. So, we will first narrow down the hypothesis to not even the European countries, but specific to London and Berlin. 
- There were market with rent 0 (WUPLO, REWEONLINEMARKET) which are row 29,30 on the initial data file that turned out to be an online market that does not have a rent. Thus, I thought it did not meet the purpose of the assignment as we are comparing the physical markets, I dropped the rows. 
- Convert the place of stay (there are many expressions for the same place)

### What can also be done with the given data
- In the future, maybe depending on the category of what type of market it is(cheap, mid-range, luxury), only check if the hypothesis is valid inside that category. Because, I can easily think of the cases when luxury goods are sold on expensive places, but those on cheap rents, may still sell goods in expensive prices as it is already marked as 'luxury' 
- Thus, kinda similar to ANOVA tests. Which is not the main focus of the class.

### Data

In [1]:
#prepare the environment
import pandas as pd
import pystan
import scipy.stats as sts
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns

#ignore the below comment
#compiler=mingw32.exe

In [2]:
#import the data
#_q for quantity
#_p for price

data = pd.read_csv(r'C:\Users\green\Desktop\data.csv', 
                   header = 2, names = ['Timestamp', 'Email', 'Your name', 'Country', 'Currency', 'Store name', 'Store type', 'Rental Price', 
                                        'Apple1_q', 'Apple1_p','Apple2_q', 'Apple2_p', 'Apple3_q', 'Apple3_p',
                                        'Banana1_q', 'Banana1_p','Banana2_q','Banana2_p','Banana3_q', 'Banana3_p',
                                        'Tomatoes1_q','Tomatoes1_p','Tomatoes2_q','Tomatoes2_p', 'Tomatoes3_q', 'Tomatoes3_p',
                                        'Potato1_q','Potato1_p', 'Potato2_q','Potato2_p','Potato3_q','Potato3_p',
                                        'Flour1_q','Flour1_p','Flour2_q','Flour2_p', 'Flour3_q','Flour3_p',
                                        'Rice1_q','Rice1_p','Rice2_q','Rice2_p', 'Rice3_q','Rice3_p',
                                        'Milk1_q','Milk1_p','Milk2_q','Milk2_p', 'Milk3_q', 'Milk3_p',
                                        'Butter1_q', 'Butter1_p', 'Butter2_q', 'Butter2_p', 'Butter3_q', 'Butter3_p',
                                        'Eggs1_q','Eggs1_p','Eggs2_q', 'Eggs2_p','Eggs3_q','Eggs3_p',
                                        'Chicken1_q', 'Chicken1_p','Chicken2_q','Chicken2_p','Chicken3_q','Chicken3_p'
                                          ]
                     )
print(data.shape)
print(data.head())


(48, 68)
             Timestamp                       Email        Your name  Country  \
0  10-24-2021 16:05:36  felipehlvo@uni.minerva.edu  Felipe Oliveira  Germany   
1  10-24-2021 16:15:08  felipehlvo@uni.minerva.edu  Felipe Oliveira  Germany   
2  10-24-2021 22:14:32    jeongwoo@uni.minerva.edu   Jeong woo Park    Seoul   
3  10-26-2021 18:31:01   anglilian@uni.minerva.edu      Ang Li-Lian   London   
4  10-26-2021 18:31:52   anglilian@uni.minerva.edu      Ang Li-Lian   London   

     Currency                                         Store name  \
0         EUR                           EDEKA, Kottbusser Damm 5   
1         EUR                         EDEKA, Pannierstr. 37 - 38   
2  Korean Won            롯데마트(lotte mart), 서울특별시 중구 회현동 한강대로 405   
3         GBP  Waitrose & Partners, 21 Liverpool Rd, The Ange...   
4         GBP  Sainsbury's, 31-41 Liverpool Rd, The Angel, Lo...   

           Store type                                       Rental Price  \
0           Mid-range    

In [3]:
#drop the unnecessary columns
data = data.drop(labels = ['Timestamp', 'Email', 'Your name', 'Store name'], axis = 1)
#drop the unnecessary rows
data = data.drop(2)#dropping Korea 
data = data.drop(25)#dropping the internet shopping(zeineb)
data = data.drop(26)#dropping the internet shopping(zeineb)
print(data.shape)
print(data.head())


(45, 64)
           Country Currency          Store type Rental Price  Apple1_q  \
0          Germany      EUR           Mid-range          924       1.0   
1          Germany      EUR           Mid-range          941       1.0   
3           London      GBP  Luxury (expensive)         2150       1.0   
4           London      GBP           Mid-range         2150       1.0   
5  United Kingdom       GBP      Budget (cheap)        2,150       1.0   

   Apple1_p  Apple2_q  Apple2_p  Apple3_q  Apple3_p  ...  Eggs2_q  Eggs2_p  \
0      2.99       1.0      2.49       1.0     4.990  ...     10.0     1.59   
1      2.99       1.0      2.99       1.0     2.990  ...      6.0     2.49   
3      2.20       0.1      0.50       0.1     0.625  ...      1.0    29.20   
4      1.95       0.1     38.30       NaN       NaN  ...      1.0    30.00   
5      1.95       1.0      1.76       1.0     1.980  ...      1.0     0.29   

   Eggs3_q  Eggs3_p  Chicken1_q  Chicken1_p  Chicken2_q  Chicken2_p  \
0     

In [4]:
#Data Formatting1 (Make the prices to the same quantity and have the prices into one currency-USD)
for i in range(5,65,2): #the index numbers of the price columns, total 67 before the drop.
    data[data.columns[i]] /= data[data.columns[i-1]]
#dropping the qantities
columns_to_drop = [i for i in range(4,64,2)]
data_formatted = data.drop(data.columns[columns_to_drop], axis = 1)
#make currencies the same with USD
print(data_formatted.Currency.unique())
exchange_rate = []
for currency in data_formatted.Currency:
    if currency == 'EUR':
    #Current Euro currency rate to USD: 1.16USD = 1euro
        exchange_rate.append(1/1.16)
    elif currency == 'GBP':
    #Current Pound currency rate to USD
        exchange_rate.append(1/1.36)
    else:
        exchange_rate.append(1)
        
#print(len(exchange_rate))

data_formatted['Currency Conversion Rate'] = exchange_rate

#check if things are done well
print(data_formatted.head())
print(data_formatted.shape) #from here we find the last number of the column 'k'

#Now multiply the prices with the conversion rate
for i in range(4,34):
    data_formatted[data_formatted.columns[i]] *= data_formatted[data_formatted.columns[34]]



['EUR' 'GBP' 'USD']
           Country Currency          Store type Rental Price  Apple1_p  \
0          Germany      EUR           Mid-range          924      2.99   
1          Germany      EUR           Mid-range          941      2.99   
3           London      GBP  Luxury (expensive)         2150      2.20   
4           London      GBP           Mid-range         2150      1.95   
5  United Kingdom       GBP      Budget (cheap)        2,150      1.95   

   Apple2_p  Apple3_p  Banana1_p  Banana2_p  Banana3_p  ...  Butter1_p  \
0      2.49      4.99       0.88   1.990000        NaN  ...       6.76   
1      2.99      2.99       0.88   1.990000        NaN  ...       9.16   
3      5.00      6.25       0.83   2.113333       1.79  ...       8.80   
4    383.00       NaN       0.73        NaN        NaN  ...       7.40   
5      1.76      1.98       0.73   1.730000       1.72  ...       5.92   

   Butter2_p  Butter3_p    Eggs1_p  Eggs2_p    Eggs3_p  Chicken1_p  \
0      13.16       9

In [5]:
#Now Treat the country names
#first what are the countries in the data?
print(data_formatted["Country"].unique())
data_formatted["Country"] = data_formatted["Country"].replace(["Germany", "Germany ", "Berlin, Germany", "Berlin, Germany ", "Berlin", "Berlin ", "UK", "United Kingdom", "United Kindom", "United Kingdom "], ["Berlin", "Berlin","Berlin", "Berlin","Berlin","Berlin", "London","London", "London", "London"])
#check if things are well formatted
print(data_formatted["Country"].unique())

['Germany' 'London' 'United Kingdom ' 'United Kingdom' 'UK'
 'Berlin, Germany' 'Germany ' 'United Kindom' 'United States' 'Berlin'
 'Berlin ']
['Berlin' 'London' 'United States']


In [6]:
#Check the Store types
data_formatted["Store type"].unique()

#Now take a look at the data that is being prepared to work with
print(data_formatted.shape)
data_formatted.head()

data_correlation = data_formatted
print(data_correlation.head())
#Now we don't need the columns with the currency and currency converter
data_formatted = data_formatted.drop(labels = ['Currency', 'Currency Conversion Rate'], axis =1)

print(data_correlation["Rental Price"].unique)
data_correlation["Rental Price"] = data_correlation["Rental Price"].replace(['2,150', '1,050', '1600 for 1 bedroom app.', ' 1600 -- for 1 bedroom apartment (+ living room + kitchen)'], [2150, 1050, 1600, 1600])
print(len(data_correlation))
data_correlation


(45, 35)
  Country Currency          Store type Rental Price  Apple1_p    Apple2_p  \
0  Berlin      EUR           Mid-range          924  2.577586    2.146552   
1  Berlin      EUR           Mid-range          941  2.577586    2.577586   
3  London      GBP  Luxury (expensive)         2150  1.617647    3.676471   
4  London      GBP           Mid-range         2150  1.433824  281.617647   
5  London      GBP      Budget (cheap)        2,150  1.433824    1.294118   

   Apple3_p  Banana1_p  Banana2_p  Banana3_p  ...  Butter1_p  Butter2_p  \
0  4.301724   0.758621   1.715517        NaN  ...   5.827586  11.344828   
1  2.577586   0.758621   1.715517        NaN  ...   7.896552   7.896552   
3  4.595588   0.610294   1.553922   1.316176  ...   6.470588   5.147059   
4       NaN   0.536765        NaN        NaN  ...   5.441176   4.352941   
5  1.455882   0.536765   1.272059   1.264706  ...   4.352941   4.852941   

   Butter3_p    Eggs1_p    Eggs2_p    Eggs3_p  Chicken1_p  Chicken2_p  \
0   

,Country,Currency,Store type,Rental Price,Apple1_p,Apple2_p,Apple3_p,Banana1_p,Banana2_p,Banana3_p,...,Butter1_p,Butter2_p,Butter3_p,Eggs1_p,Eggs2_p,Eggs3_p,Chicken1_p,Chicken2_p,Chicken3_p,Currency Conversion Rate
0,Berlin,EUR,Mid-range,924,2.577586,2.146552,4.301724,0.758621,1.715517,NaN,...,5.827586,11.344828,7.896552,0.369828,0.137069,0.285920,25.775862,21.543103,12.844828,0.862069
1,Berlin,EUR,Mid-range,941,2.577586,2.577586,2.577586,0.758621,1.715517,NaN,...,7.896552,7.896552,7.551724,0.429598,0.357759,0.240517,6.448276,NaN,NaN,0.862069
3,London,GBP,Luxury (expensive),2150,1.617647,3.676471,4.595588,0.610294,1.553922,1.316176,...,6.470588,5.147059,4.117647,0.373529,21.470588,NaN,6.433824,5.514706,5.514706,0.735294
4,London,GBP,Mid-range,2150,1.433824,281.617647,NaN,0.536765,NaN,NaN,...,5.441176,4.352941,5.294118,12.279412,22.058824,20.220588,3.669118,NaN,NaN,0.735294
5,London,GBP,Budget (cheap),2150,1.433824,1.294118,1.455882,0.536765,1.272059,1.264706,...,4.352941,4.852941,5.441176,0.132353,0.213235,0.360294,4.654412,NaN,NaN,0.735294
6,London,GBP,Mid-range,2150,1.470588,2.058824,1.617647,0.610294,2.330882,1.470588,...,4.705882,5.147059,6.176471,0.125000,0.294118,0.220588,5.514706,5.941176,12.727941,0.735294
7,Berlin,EUR,Mid-range,942,2.577586,2.577586,2.146552,0.853448,1.715517,0.758621,...,6.862069,8.931034,7.551724,0.240517,0.429598,0.357759,6.456897,12.922414,21.543103,0.862069
8,Berlin,EUR,Mid-range,794,2.577586,2.146552,2.577586,0.767241,1.715517,0.853448,...,6.517241,7.551724,8.931034,0.357759,0.240517,0.429598,6.198276,13.698276,19.215517,0.862069
9,London,GBP,Mid-range,1846,3.460208,NaN,NaN,1.531863,1.021242,NaN,...,1.470588,3.455882,6.176471,0.128676,NaN,NaN,12.975779,7.257448,12.132353,0.735294
10,London,GBP,Mid-range,1846,2.941176,2.941176,4.757785,1.531863,1.531863,NaN,...,2.941176,3.235294,5.882353,0.116422,0.137868,NaN,11.233660,9.803922,NaN,0.735294


In [7]:
#The variables for the prices
variables_p = ['Apple1_p', 'Apple2_p', 'Apple3_p',
         'Banana1_p', 'Banana2_p', 'Banana3_p',
         'Tomatoes1_p', 'Tomatoes2_p', 'Tomatoes3_p',
         'Potato1_p', 'Potato2_p', 'Potato3_p',
         'Flour1_p', 'Flour2_p', 'Flour3_p',
         'Rice1_p', 'Rice2_p', 'Rice3_p',
         'Milk1_p', 'Milk2_p', 'Milk3_p',
         'Butter1_p', 'Butter2_p', 'Butter3_p',
         'Eggs1_p', 'Eggs2_p', 'Eggs3_p',
         'Chicken1_p', 'Chicken2_p', 'Chicken3_p'
         ]
#Melt the data_formatted on price
data_formatted_new = pd.melt(data_formatted, id_vars = ['Country', 'Store type'], 
                               value_vars= variables_p, 
                               value_name = 'Price', 
                               var_name = 'Product')
#now see the number of rows and columns
print(data_formatted_new.head())
data_formatted_new.shape

  Country          Store type   Product     Price
0  Berlin           Mid-range  Apple1_p  2.577586
1  Berlin           Mid-range  Apple1_p  2.577586
2  London  Luxury (expensive)  Apple1_p  1.617647
3  London           Mid-range  Apple1_p  1.433824
4  London      Budget (cheap)  Apple1_p  1.433824


(1350, 4)

In [8]:
#Change the column names of the products to contain 10 keywords for each of the product type
data_formatted_new["Product"] = data_formatted_new["Product"].apply(lambda x: re.sub(r'[0-9]{1}_p', '',x))

#Variable Names as arrays
countries = data_formatted_new["Country"].unique()
store_types = data_formatted_new["Store type"].unique()
products = data_formatted_new["Product"].unique()

#Now Exchange the category names
data_formatted_new["Country"] = data_formatted_new["Country"].replace(["United States", "London", "Berlin"], [1,2,3])
data_formatted_new["Store type"] = data_formatted_new["Store type"].replace(['Mid-range', 'Luxury (expensive)', 'Budget (cheap)'],[1,2,3])
data_formatted_new["Product"] = data_formatted_new["Product"].replace([
    'Apple', 'Banana', 'Tomatoes', 'Potato', 'Flour', 'Rice', 'Milk', 'Butter', 'Eggs', 'Chicken'],[1,2,3,4,5,6,7,8,9,10])

#check whether the translations are done correctly
print(data_formatted_new["Country"].unique())
print(data_formatted_new["Store type"].unique())
print(data_formatted_new["Product"].unique())
print(data_formatted_new)
#Now the data is all prepared for use for stan

[3 2 1]
[1 2 3]
[ 1  2  3  4  5  6  7  8  9 10]
      Country  Store type  Product     Price
0           3           1        1  2.577586
1           3           1        1  2.577586
2           2           2        1  1.617647
3           2           1        1  1.433824
4           2           3        1  1.433824
...       ...         ...      ...       ...
1345        3           3       10       NaN
1346        3           1       10       NaN
1347        3           3       10       NaN
1348        3           1       10       NaN
1349        3           1       10  4.525862

[1350 rows x 4 columns]


In [9]:
#there is a nan in the data. It seems to be better to run without it rather than filling in a number
#I was initially thinking of putting 0 instead of NAN or mean price of that product, 
#but that might change the distribution shape, and thus I chose to drop NAN rows
data_formatted_new = data_formatted_new.dropna()
print(data_formatted_new)

#Prepare the Stan
model_data = {
    
    #Data values
    "prices_N" : data_formatted_new.shape[0],
    'prices' : data_formatted_new["Price"].values, #price values

    'store_N': len(data_formatted_new["Store type"].unique()),
    "store_types" : data_formatted_new["Store type"].values, #3 types of stores

    'country_N': len(data_formatted_new["Country"].unique()),
    "countries" : data_formatted_new["Country"].values, #countries 

    'products_N': len(data_formatted_new["Product"].unique()),  
    'products': data_formatted_new["Product"].values #10 types of products

}
stan_code = """

data {
    int<lower=1> prices_N;                  // number of observed prices
    real<lower =0> prices[prices_N];         //list of prices


    int<lower=1> store_N;                 // number of different store types
    int<lower =1> store_types[prices_N];            // list of store types


    int<lower=1> country_N;               // number of countries 
    int<lower = 1> countries[prices_N];  // list of countries 


    int<lower = 1> products_N;            // number of different products presented
    int<lower = 1> products[prices_N];  // list of different products

}


parameters {

    // 3 multipliers to input into likelihood function
    real<lower = 0> base_price[products_N]; // 10 different base prices
    real<lower=0> country_multiplier[country_N]; // 6 different multipliers
    real<lower = 0> store_multiplier[store_N]; // 3 multipliers



    // unknown parameters

    real<lower = 0> sigma;
    real<lower=0> lambda;
}

model {

    //Hyperparameters priors

    sigma ~ gamma(1,1); //uninformed prior about the uncertainty of the N distribution
    lambda ~ gamma(1, 0.2); //hyperparameter for the exponential likelihood function


    // Priors

    base_price ~ exponential(lambda);
    country_multiplier ~ lognormal(0,1);
    store_multiplier ~ lognormal(0,1);


    //Likelihood
    for (datum in 1:prices_N) {
        prices[datum] ~ normal(base_price[products[datum]]*country_multiplier[countries[datum]]*store_multiplier[store_types[datum]], sigma);
    }
}
"""
stan_model = pystan.StanModel(model_code=stan_code)
results = stan_model.sampling(data=model_data)
print(results)
samples = results.extract()

      Country  Store type  Product      Price
0           3           1        1   2.577586
1           3           1        1   2.577586
2           2           2        1   1.617647
3           2           1        1   1.433824
4           2           3        1   1.433824
...       ...         ...      ...        ...
1338        3           1       10  10.336207
1339        3           3       10   6.724138
1340        3           1       10   7.396552
1341        3           3       10   7.094828
1349        3           1       10   4.525862

[1180 rows x 4 columns]


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_a6ee1f88cf447bd0f0da56982ec33a90 NOW.


TypeError: '>=' not supported between instances of 'NoneType' and 'str'

In [ ]:
#From OREILLY text book of "Data Visualization Handbook"

#Predicted Prices of Products itself (not considering the country yet)
plt.figure(figsize=(15,6))

for i in range(model_data['products_N']): #for each of the product
    plt.plot(sts.uniform.rvs(loc=i, scale=0.4, size = 4000),
             samples['base_price'][:,i], ',')

plt.title('Samples of posterior distributions in pixels (scatter plot) of base prices')
plt.xticks(range(1,11), (products))
plt.ylim(0, 30)
plt.show()

In [ ]:
#Now we multiply the country type's weight
#Then, what is the distribution of the prices of goods on each country?
#Show
plt.figure(figsize=(15,6))

for i in range(model_data['country_N']): #for each of the country
    plt.plot(sts.uniform.rvs(loc=i, scale=0.4, size = 4000),
             samples['country_multiplier'][:,i], ',')


plt.title('Samples of posterior distributions of base prices')
plt.xticks(range(1,4), (countries))
plt.ylim(0, 4)
plt.show()

In [ ]:
#Again, similarly with the type of the store we need to find the distribution of prices
#Show
plt.figure(figsize=(15,6))

for i in range(model_data['store_N']): #for each of the store type
    plt.plot(sts.uniform.rvs(loc=i, scale=0.4, size = 4000),
             samples['country_multiplier'][:,i], ',')

plt.title('Samples of posterior distributions of store types')
plt.xticks(range(1,4), (store_types))
plt.ylim(0, 10)
plt.show()

In [ ]:
#Now the correlations between the rental prices and the product goods
#First prepare the data
#country multipliers data

#creating df containing country and the rental price
rental_df_data = [data_correlation["Rental Price"],data_correlation["Country"], data_correlation["Currency Conversion Rate"]]
rental_df_headers = ['Rental_prices', 'Country', "Currency_Converter"]
rental_price = pd.concat(rental_df_data, axis =1, keys = rental_df_headers)
print(rental_price.head())

#converting the prices to dollars
rental_price.Rental_prices= pd.to_numeric(rental_price.Rental_prices, downcast="float")
rental_price.Rental_prices *=rental_price.Currency_Converter

#get the prices based on a country
USA = []
London = []
Berlin = []

for i in range (len(rental_price)):
    #sort out US rental prices
    if rental_price.loc[i,'Country'] == 'United States':
        USA.append(rental_price.loc[i,'Rental_prices'])

    elif rental_price.loc[i,'Country'] == 'London':
        London.append(rental_price.loc[i,'Rental_prices'])


    elif rental_price.loc[i,'Country'] == 'Berlin':
        Berlin.append(rental_price.loc[i,'Rental_prices'])

#mean values of the rent
mean = [np.mean(USA),np.mean(London), np.mean(Berlin)]

#Correlation Caluclation(From CS112)
correlation_matrix = np.corrcoef(means, countries_multipliers)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2 

print(r_squared)


# regression plot using seaborn
fig = plt.figure(figsize=(15, 8))
sns.regplot(x=means, y=countries_multipliers, color='blue', marker='+')

# Legend, title and labels.
plt.title('Relationship between Rental Prices and Country Multipliers', size=24)
plt.xlabel('Mean Values for the Rental Prices in Different countries', size=18)
plt.ylabel('Country Multipliers', size=18);

